# Extracting Data from www.kaggle.com and preprocessing and saving onto a MongoDB database

In [3]:
# import libraries
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient
import certifi
from mongo2_config import mongo_uri

In [4]:
# create a dataframe from the csv
path ="C:/UC_Berkeley\Analysis_Projects_Class_Folder/Module20_Final_Project\Data_Kaggle_Historical/SPX.csv"
df = pd.read_csv(path)
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0
...,...,...,...,...,...,...,...
23318,2020-10-29,3277.169922,3341.050049,3259.820068,3310.110107,3310.110107,4903070000
23319,2020-10-30,3293.590088,3304.929932,3233.939941,3269.959961,3269.959961,4840450000
23320,2020-11-02,3296.199951,3330.139893,3279.739990,3310.239990,3310.239990,4310590000
23321,2020-11-03,3336.250000,3389.489990,3336.250000,3369.159912,3369.159912,4220070000


In [6]:
# checking for mull values
df.isnull().sum().sum()
df.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
# converting date to datetime format
df['Date'] = pd.to_datetime(df['Date'])

In [8]:
# data being analyzed for last five years, so filtering data for that date range
start = '2016-07-06'
end = '2021-07-06'

In [9]:
# creating a variable to store that date range
fiveyears = (df['Date']>start) & (df['Date']<= end)

In [10]:
# creating a new dataframe after applying the above filters
S_P_df = df.loc[fiveyears]
S_P_df

,Date,Open,High,Low,Close,Adj Close,Volume
22231,2016-07-07,2100.419922,2109.080078,2089.389893,2097.899902,2097.899902,3604550000
22232,2016-07-08,2106.969971,2131.709961,2106.969971,2129.899902,2129.899902,3607500000
22233,2016-07-11,2131.719971,2143.159912,2131.719971,2137.159912,2137.159912,3253340000
22234,2016-07-12,2139.500000,2155.399902,2139.500000,2152.139893,2152.139893,4097820000
22235,2016-07-13,2153.810059,2156.449951,2146.209961,2152.429932,2152.429932,3502320000
...,...,...,...,...,...,...,...
23318,2020-10-29,3277.169922,3341.050049,3259.820068,3310.110107,3310.110107,4903070000
23319,2020-10-30,3293.590088,3304.929932,3233.939941,3269.959961,3269.959961,4840450000
23320,2020-11-02,3296.199951,3330.139893,3279.739990,3310.239990,3310.239990,4310590000
23321,2020-11-03,3336.250000,3389.489990,3336.250000,3369.159912,3369.159912,4220070000


In [11]:
# exporting the dataframe to csv
S_P_df.to_csv('S_P(fiveyears).csv')

# Storing file in MongoDB (DB name : Finale)

In [12]:
# Read in mongodb server location as client
client = MongoClient(mongo_uri,tlsCAFile=certifi.where())

In [13]:
# Find Databases
client.list_database_names()

['Kaggle_data', 'Latest_Cryptodata', 'Latest_StockMarket', 'admin', 'local']

In [17]:
# connecting to the db on MongoDb and creating collection in the db
db = client['Kaggle_data']
collection = db['S_P_fiveyears']

In [18]:
# adding dataframe to mongoDb
S_P_df.reset_index(inplace=True)
S_P_df_dict = S_P_df.to_dict("records")

In [19]:
# Insert collection
collection.insert_many(S_P_df_dict)